# Getting Started

### Loading the dataset 

In [1]:
from scipy.io import wavfile
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [13]:
class SongLoader:
    
    def __init__(self):
        self.paths = []
        
    def append(self, x):
        self.paths.append(x)
    
    def __getitem__(self, ind):
        _, data = wavfile.read(self.paths[ind])
        return data
    
    def concat_data(self):
        out = []
        for path in self.paths:
            _, data = wavfile.read(path)
            out.append(data[:7000000])
        return np.vstack(out)

    
def loadDSD100(path):
    mixPath = path + "/Mixtures/"
    sourcePath = path + "/Sources/"
    instruments = ["bass", "drums", "vocals", "other"]
    out = { "instruments" : instruments,
            "fs":  int(44.1 * 10e3), 
            "Xtrain": SongLoader(),  
            "Ytrain": {}, 
            "Xtest" : SongLoader(),
            "Ytest" : {} }

    for inst in instruments:
        out["Ytest"][inst] = SongLoader()
        out["Ytrain"][inst] = SongLoader()
    
    for dtype in ["Dev", "Test"]:
        Xmode, Ymode = ["Xtrain", "Ytrain"] if dtype == "Dev" else ["Xtest", "Ytest"]
        for file in os.listdir(mixPath + dtype):
            out[Xmode].append(mixPath + dtype + "/" + file + "/mixture.wav")
        
        for file in os.listdir(sourcePath + dtype):
            for inst in instruments: 
                out[Ymode][inst].append(sourcePath + dtype + "/" + file + "/" + inst + ".wav")
    return out 
g

### Plot DFTs 

In [ ]:
def plotDFT(x, title): 
    fig, axs = plt.subplots(2)
    N = x.shape[0]
    fs = data["fs"]
    time = np.arange(N) / fs
    if N % 2:
        N = N + 1
    freq = fs * np.linspace(0, N//2, N//2+1, dtype=float) / N
    Y = np.fft.fft(x, N, axis=0)
    axs[0].plot(time ,x )
    mags = abs(Y[0:N//2+1])
    axs[1].plot(freq, np.log(mags))

plotDFT(data["Xtrain"][1][:,0], "mix")   



### Spectrograms

In [ ]:
def plotSpectogram(x, title):
    N = x.shape[0]
    fs = data["fs"]
    f, t, Sxx = signal.stft(x, fs ,nperseg = 10000) #, nperseg = npseg, noverlap = npseg//8
    plt.title(title)
    plt.pcolormesh(t, f, np.log(np.abs(Sxx) + 1e-15), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

plotSpectogram(data["Xtrain"][4][:,0], "mix")   
for k in data["instruments"]: 
    plotSpectogram(data["Ytrain"][k][4][:,0], k)

### ICA

In [14]:
from sklearn.decomposition import FastICA
ica = FastICA(n_components = 2)
X = data["Xtrain"].concat_data()
print(X.shape)


(350000000, 2)


In [9]:
S_ = ica.fit_transform(X)

print(S_.shape)


(12163922, 2)
